In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data_steven_anon.csv')

In [3]:
df = df[df.institute_abbreviation == 'CWZ']

In [4]:
import ast
def load_visual_features(path):
    visual_features = pd.read_csv(path)
    print(len(visual_features))
#     print(visual_features)
    visual_features.features = visual_features.features.str.replace(r'[^\[] +', ', ', regex=True).apply(ast.literal_eval)
    vf = visual_features.features.apply(pd.Series)
    vf.columns = vf.columns.map('vis_feature_{}'.format)
    visual_features = pd.concat([visual_features[['scan_id']], vf], axis=1)
#     visual_features['patientprimarymrn'] = pd.to_numeric(visual_features.scan_id.str.extract(r'(\d+)_st\d+').iloc[:, 0])
#     visual_features['study'] = visual_features.scan_id.str.extract(r'\d+_(st\d+)').iloc[:, 0]
#     visual_features.drop(columns=['scan_id'], inplace=True)
    visual_features.set_index(['scan_id'], inplace=True)
    return visual_features

In [5]:
visual = load_visual_features('../../data/raw/cwz_preds (5).csv')
visual

258


,vis_feature_0,vis_feature_1,vis_feature_2,vis_feature_3,vis_feature_4,vis_feature_5,vis_feature_6,vis_feature_7,vis_feature_8,vis_feature_9,...,vis_feature_50,vis_feature_51,vis_feature_52,vis_feature_53,vis_feature_54,vis_feature_55,vis_feature_56,vis_feature_57,vis_feature_58,vis_feature_59
scan_id,,,,,,,,,,,,,,,,,,,,,
55,-4.496160,-2.745639,-2.575471,-0.008700,7.113443,7.396276,9.854604,0.100944,1.999237,1.616565,...,-1.405401,-1.384378,-1.212076,-1.189800,-1.187900,-1.173055,-9.122608,-8.925362,-8.849688,-0.923273
84,2.806867,2.819759,2.317768,2.330266,3.381526,3.504354,2.904047,3.024475,3.187248,3.503367,...,2.841778,2.728686,0.822111,0.793279,0.873804,0.829842,-0.390464,-0.386274,-0.371973,-0.413959
29,-0.580699,-0.551752,-0.558355,-0.530318,-0.433520,-0.509670,-0.406444,-0.483075,0.189461,-0.107519,...,-1.202362,-1.164445,-0.970110,-0.947747,-0.973538,-0.955477,-0.695281,-0.698520,-0.711876,-0.763410
171,0.184839,0.222050,0.176674,0.214625,1.332345,1.368712,1.321980,1.359175,2.745847,2.716447,...,0.770675,0.628775,-0.433643,-0.450494,-0.448619,-0.468479,-0.662406,-0.665938,-0.691295,-0.743963
261,0.359975,0.414118,0.336472,0.387573,0.378910,0.404797,0.372611,0.395199,0.263762,0.246561,...,-1.502049,-1.496172,-1.125306,-1.128514,-1.125185,-1.131577,-0.706025,-0.720274,-0.715704,-0.769694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,0.064539,0.080916,0.090361,0.103957,1.185347,1.146651,1.205684,1.164970,2.591774,2.446881,...,4.087820,4.101952,1.242651,1.254284,1.233551,1.241323,-0.322520,-0.331646,-0.340867,-0.399153
140,-0.413619,-0.457544,-0.454490,-0.487602,-0.627621,-0.645841,-0.629948,-0.639792,-0.408087,-0.303671,...,-1.531370,-1.509033,-1.252813,-1.244135,-1.237327,-1.231619,-0.864253,-0.871522,-0.867404,-0.917374
257,-0.510884,-0.469759,-0.484806,-0.444568,-0.706197,-0.655956,-0.676428,-0.626828,-0.874422,-0.834990,...,-1.136543,-1.103573,-0.910307,-0.888976,-0.910678,-0.893885,-0.633696,-0.634708,-0.647933,-0.699448


In [6]:
df['scan_id'] = pd.to_numeric(df.record_id.str.split('-', expand=True).iloc[:, 2])

In [7]:
df = df.join(visual, on='scan_id')

In [8]:
visual_features = df.columns[df.columns.str.startswith('vis')]
clinical_features = (['dem_pat_age_at_inclusion', 'dem_pat_gender', 'dem_pat_BMI'] 
                    + list(df.columns[df.columns.str.startswith('med_hist')].values)
                    + list(df.columns[df.columns.str.startswith('adm') & ~df.columns.isin(['adm_date', 'adm_dept', 'adm_hosp_date'])].values))

In [47]:
X = df[list(clinical_features)+list(visual_features)]
y = df['COVID_and_prob_and_pos']

In [48]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from lightgbm.sklearn import LGBMClassifier

In [74]:
cross_val_score(LGBMClassifier(n_jobs=-1, boosting_type='gbdt'), X, y, scoring='roc_auc', cv=StratifiedKFold(n_splits=30, shuffle=True)).mean()

0.9087301587301587

In [82]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [58]:
import re
class PasswordFilter(object):
    def __init__(self, strings_to_filter, stream):
        self.stream = stream
        self.strings_to_filter = strings_to_filter

    def __getattr__(self, attr_name):
        return getattr(self.stream, attr_name)

    def write(self, data):
        if data.strip() == '' and '\n' in data:
            return
        for string in self.strings_to_filter:
            if string in data:
                return
        self.stream.write(data)
        self.stream.flush()

    def flush(self):
        self.stream.flush()

In [59]:
import sys
old_out = sys.stdout

In [60]:
filter_words = ['[LightGBM]']
sys.stdout = PasswordFilter(filter_words, old_out)

In [83]:
import lightgbm as lgb

import optuna
import numpy as np

from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
import warnings

cv = RepeatedStratifiedKFold(n_repeats=3, n_splits=16)

# 1. Define an objective function to be maximized.
def objective(trial):
    # 2. Suggest values of the hyperparameters using a trial object.
    param = {
        'objective': 'binary',
        'metric': 'auc',
        'verbosity': -1,
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart', 'rf', 'goss']),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-10, 100.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-10, 100.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 400),
        'max_depth': trial.suggest_int('max_depth', 0, 40), 
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.01, 1.0),
        'feature_fraction_bynode': trial.suggest_uniform('feature_fraction_bynode', 0.01, 1.0), 
        'min_child_samples': trial.suggest_int('min_child_samples', 2, 200),
        'linear_tree': trial.suggest_categorical('linear_tree', [False, True]), 
        'feature_pre_filter': False, 
        'max_bin': trial.suggest_int('max_bin', 7, 511), 
        'extra_trees': trial.suggest_categorical('extra_trees', [False, True]), 
        'path_smooth': trial.suggest_loguniform('path_smooth', 1e-12, 100), 
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-30, 1), 
        'min_gain_to_split': trial.suggest_loguniform('min_gain_to_split', 1e-20, 100), 
        'min_sum_hessian_in_leaf': trial.suggest_loguniform('min_sum_hessian_in_leaf', 1e-6, 1e2), 
        'num_boost_round': trial.suggest_int('num_boost_round', 2, 600)
    }
    
    if param['boosting_type'] != 'goss':
        not_goss_params = {
            'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.01, 0.999999),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 100),
        }
        param.update(not_goss_params)

    if param['boosting_type'] == 'dart':
        dart_params = {
          'drop_rate': trial.suggest_uniform('drop_rate', 0.01, 1.0), 
          'max_drop': trial.suggest_int('max_drop', 0, 200), 
          'skip_drop': trial.suggest_uniform('skip_drop', 0.001, 1.0), 
          'xgboost_dart_mode': trial.suggest_categorical('xgboost_dart_mode', [False, True]), 
          'uniform_drop': trial.suggest_categorical('uniform_drop', [False, True])
        }

        param.update(dart_params)
    
    clf = LGBMClassifier(n_jobs=-1, **param)
#     clf = BaggingClassifier(LGBMClassifier(n_jobs=-1, **param), n_estimators=55, 
#                             bootstrap=trial.suggest_categorical('bootstrap', [False, True]), 
#                             bootstrap_features=trial.suggest_categorical('bootstrap_features', [False, True]), 
#                            max_samples=trial.suggest_uniform('max_samples', 0.01, 1.0), 
#                            max_features=trial.suggest_uniform('max_features', 0.01, 1.0))
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore')
        return cross_val_score(clf, X_train, y_train, cv=cv, scoring='roc_auc').mean()
#     aucs = []
#     for i in range(5):
#         train_data = lgb.Dataset(imputer.transform(X_train), label=y_train)
#         aucs.append(lgb.cv(param, train_data, num_boost_round=num_boost_round, nfold=7)['auc-mean'][-1])
#     # auc = roc_auc_score(split_data(test)[1], bst.predict(split_data(test)[0]))
#     return np.mean(aucs)

In [85]:
# 3. Create a study object and optimize the objective function.
study = optuna.create_study(direction='maximize')

[I 2021-06-01 12:35:34,182] A new study created in memory with name: no-name-cb2c5bb2-082b-434c-9bd3-f287605ecb2b


In [87]:
study.enqueue_trial({'boosting_type': 'dart',
 'lambda_l1': 0.4730954965398153,
 'lambda_l2': 0.003396007397622806,
 'num_leaves': 224,
 'max_depth': 24,
 'feature_fraction': 0.5895327448818392,
 'feature_fraction_bynode': 0.7987049600837334,
 'min_child_samples': 7,
 'linear_tree': False,
 'max_bin': 71,
 'extra_trees': True,
 'path_smooth': 0.0607767028476868,
 'learning_rate': 0.25826315149322404,
 'min_gain_to_split': 1.0001105239111538e-05,
 'min_sum_hessian_in_leaf': 0.005150076006918998,
 'num_boost_round': 327,
 'bagging_fraction': 0.9648862602646029,
 'bagging_freq': 36,
 'drop_rate': 0.7372301551559457,
 'max_drop': 82,
 'skip_drop': 0.5763271846459017,
 'xgboost_dart_mode': False,
 'uniform_drop': True})

c:\users\roel\.virtualenvs\diagcovid-ckc2berq\lib\site-packages\ipykernel_launcher.py:23: ExperimentalWarning: enqueue_trial is experimental (supported from v1.2.0). The interface can change in the future.
c:\users\roel\.virtualenvs\diagcovid-ckc2berq\lib\site-packages\optuna\study.py:857: ExperimentalWarning: create_trial is experimental (supported from v2.0.0). The interface can change in the future.
  create_trial(state=TrialState.WAITING, system_attrs={"fixed_params": params})
c:\users\roel\.virtualenvs\diagcovid-ckc2berq\lib\site-packages\optuna\study.py:857: ExperimentalWarning: add_trial is experimental (supported from v2.0.0). The interface can change in the future.
  create_trial(state=TrialState.WAITING, system_attrs={"fixed_params": params})


In [88]:
study.optimize(objective, n_trials=4000)

[I 2021-06-01 12:36:16,331] Trial 2 finished with value: 0.9068287037037037 and parameters: {'boosting_type': 'dart', 'lambda_l1': 0.4730954965398153, 'lambda_l2': 0.003396007397622806, 'num_leaves': 224, 'max_depth': 24, 'feature_fraction': 0.5895327448818392, 'feature_fraction_bynode': 0.7987049600837334, 'min_child_samples': 7, 'linear_tree': False, 'max_bin': 71, 'extra_trees': True, 'path_smooth': 0.0607767028476868, 'learning_rate': 0.25826315149322404, 'min_gain_to_split': 1.0001105239111538e-05, 'min_sum_hessian_in_leaf': 0.005150076006918998, 'num_boost_round': 327, 'bagging_fraction': 0.9648862602646029, 'bagging_freq': 36, 'drop_rate': 0.7372301551559457, 'max_drop': 82, 'skip_drop': 0.5763271846459017, 'xgboost_dart_mode': False, 'uniform_drop': True}. Best is trial 2 with value: 0.9068287037037037.
[I 2021-06-01 12:36:21,889] Trial 3 finished with value: 0.5 and parameters: {'boosting_type': 'goss', 'lambda_l1': 3.460985130854667e-10, 'lambda_l2': 18.221678708569655, 'num_

KeyboardInterrupt: 

In [89]:
study.best_params

{'boosting_type': 'gbdt',
 'lambda_l1': 0.0006153700322148878,
 'lambda_l2': 0.00015730524563834818,
 'num_leaves': 162,
 'max_depth': 35,
 'feature_fraction': 0.7545564618621851,
 'feature_fraction_bynode': 0.6234454231953541,
 'min_child_samples': 2,
 'linear_tree': False,
 'max_bin': 511,
 'extra_trees': True,
 'path_smooth': 7.580133740071395e-05,
 'learning_rate': 1.0286601686914492e-06,
 'min_gain_to_split': 0.09220577019007745,
 'min_sum_hessian_in_leaf': 0.00023890668576570615,
 'num_boost_round': 569,
 'bagging_fraction': 0.8009052958650527,
 'bagging_freq': 10}

In [90]:
clf = LGBMClassifier(n_jobs=-1, **study.best_params)
clf.fit(X_train, y_train)

c:\users\roel\.virtualenvs\diagcovid-ckc2berq\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


LGBMClassifier(bagging_fraction=0.8009052958650527, bagging_freq=10,
               extra_trees=True, feature_fraction=0.7545564618621851,
               feature_fraction_bynode=0.6234454231953541,
               lambda_l1=0.0006153700322148878,
               lambda_l2=0.00015730524563834818,
               learning_rate=1.0286601686914492e-06, linear_tree=False,
               max_bin=511, max_depth=35, min_child_samples=2,
               min_gain_to_split=0.09220577019007745,
               min_sum_hessian_in_leaf=0.00023890668576570615,
               num_boost_round=569, num_leaves=162,
               path_smooth=7.580133740071395e-05)

In [92]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])

0.9347826086956522

In [98]:
roc_auc_score(y_test, LGBMClassifier(n_jobs=-1).fit(X_train[clinical_features], y_train).predict_proba(X_test[clinical_features])[:, 1])

0.9565217391304348

In [94]:
roc_auc_score(y_test, X_test.mean(axis=1).fillna(X.mean(axis=1).mean()))

0.8136645962732919

In [39]:
X.mean(axis=1).isna()

4